Import the libraries

In [25]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import os
import datetime
import requests
import urllib

load_dotenv()

True

Define tools

In [43]:
from test.test_typing import Annotated


@tool("get_current_datetime")
def get_current_datetime() -> str :
    """Get current date and time"""
    return datetime.datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

@tool("convert_to_japanse_datetime_custom")
def convert_to_japanse_datetime(date_str: Annotated[str, 'date string in ISO format']) -> str :
    """Convert date string to custom japanese date time"""
    dt = datetime.datetime.fromisoformat(date_str)
    return f"{dt.year}年{dt.month}月{dt.day}日　{dt.hour}時{dt.minute}分{dt.second}秒"

# dir: Annotated[str, 'the directory of the file location, pass in /tmp for default']
@tool("download_image_from_url")
def download_image_from_url(url: Annotated[str, 'full URL of the web image']) -> dict:
    "Download an image from web and save to local device"
    directory = './tmp'
    file_name = f"{directory}/{datetime.datetime.timestamp(datetime.datetime.now())}-agent.png"
    try:
        res= urllib.request.urlretrieve(url,file_name)    
        print(res)
        return  {
            "success": True,
            "file_name": file_name
        }
    except (Exception) as e:
         return {
                "success": False,
                "error": f"{e}",
                "url": url
            }
    
    

Initialized OpenAI LLM

In [44]:
# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_KEY'),
)

tools = [
    get_current_datetime,
    convert_to_japanse_datetime,
    download_image_from_url
]

prompt = hub.pull("hwchase17/react",api_key=os.getenv('LANGSMITH_KEY'),)
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)


# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

Testing

In [50]:
response = agent_executor.invoke(input={"input": ["Download this image for me: https://hips.hearstapps.com/hmg-prod/images/balloon-flower-royalty-free-image-1703107813.jpg"]})
print("response:", response)



> Entering new AgentExecutor chain...
I need to download the image from the provided URL.  
Action: download_image_from_url  
Action Input: "https://hips.hearstapps.com/hmg-prod/images/balloon-flower-royalty-free-image-1703107813.jpg"  ('./tmp/1740237392.807722-agent.png', <http.client.HTTPMessage object at 0x000001C5468619A0>)
{'success': True, 'file_name': './tmp/1740237392.807722-agent.png'}I have successfully downloaded the image.  
Final Answer: The image has been downloaded and saved as './tmp/1740237392.807722-agent.png'.

> Finished chain.
response: {'input': ['Download this image for me: https://hips.hearstapps.com/hmg-prod/images/balloon-flower-royalty-free-image-1703107813.jpg'], 'output': "The image has been downloaded and saved as './tmp/1740237392.807722-agent.png'."}
